In [ ]:
from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# ssObjectId of object to analyse
ssoid = "8268570668335894776"

In [ ]:
# retrieve the object data via adler

# # here we use an offline SQL database which contains the observations of the sso
# fname = "../tests/data/testing_database.db"
# planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

# alternatively we can retrieve the object data directly from the RSP
planetoid = AdlerPlanetoid.construct_from_RSP(ssoid)

In [ ]:
# inspect the object
# planetoid.__dict__

In [ ]:
planetoid.SSObject.__dict__

In [ ]:
obs_r = planetoid.observations_in_filter("r")

In [ ]:
obs_r.__dict__

In [ ]:
# plot the observations
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# define the phase curve

sso_r = planetoid.SSObject_in_filter("r")

r_H = sso_r.H
r_G12 = sso_r.G12

pc = PhaseCurve(abs_mag=r_H * u.mag, phase_param=r_G12, model_name="HG12_Pen16")

In [ ]:
pc.model_function

In [ ]:
alpha = np.linspace(0, np.amax(obs_r.phaseAngle)) * u.deg
alpha

In [ ]:
red_mag = pc.ReducedMag(alpha)
red_mag

In [ ]:
# %matplotlib widget

# plot the observations with the LSST phase curve
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.plot(alpha.value, red_mag.value)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# plot the observations
x_plot = "midpointMjdTai"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# do a different phase curve fit to the data
# adler should be able to fit different models, and perform more sophisticated fits

In [ ]:
pc_fit = PhaseCurve(abs_mag=pc.abs_mag, model_name="HG")

In [ ]:
pc_fit.model_function

In [ ]:
pc_fit.FitModel(
    phase_angle=obs_r.phaseAngle * u.deg,
    reduced_mag=obs_r.reduced_mag * u.mag,
    mag_err=obs_r.magErr * u.mag,
)

In [ ]:
# %matplotlib widget

# plot the observations with the LSST phase curve
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.plot(alpha.value, pc.ReducedMag(alpha).value, label=pc.model_name)
ax1.plot(alpha.value, pc_fit.ReducedMag(alpha).value, label=pc_fit.model_name)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()

plt.show()

In [ ]:
# # now we would add our calculated values back into planetoid
# planetoid.AdlerSchema.r_H = pc_fit.abs_mag
# planetoid.AdlerSchema.r_G = pc_fit.phase_param